In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import re
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})

In [ ]:
root_folder_path = "/kaggle/input/spark-fund-investment-analysis/datasets/"
companies_path = os.path.join(root_folder_path,"companies.txt")
rounds2_path = os.path.join(root_folder_path,"rounds2.csv")
mapping_path = os.path.join(root_folder_path,"mapping.csv")
investment_path =os.path.join(root_folder_path,"Investments.xlsx")

___
#  Checkpoint 1: Data cleaning 1
___

### 1.1 Load Companies and rounds2

In [ ]:
companies = pd.read_csv(companies_path, sep='\t', encoding="ISO-8859-1")
companies.head()

In [ ]:
rounds2 = pd.read_csv(rounds2_path, encoding="ISO-8859-1")
rounds2.head()

---
**<font size = 4>Question 1. How many unique companies are present in rounds2?</font>**
--

**<ins>Approach</ins>**: Using nunique() function of Pandas DataFrame to get the unique values in each column of the dataframe. Also considering data analysis intricacies, unifying the case of the values of the column to lower case and then cross checking for unique values

In [ ]:
rounds2.count()

In [ ]:
rounds2.nunique()

In [ ]:
len(rounds2['company_permalink'].str.lower().unique())
#Nan

In [ ]:
rounds2_lower = rounds2.copy()
rounds2_lower['company_permalink'] =rounds2_lower['company_permalink'].str.lower()
rounds2[rounds2_lower.duplicated(['company_permalink'],keep=False)].head()

**<sup>Table 1.1 Case Sensitivity in comapny_permalink column of rounds2</sup>**

In [ ]:
rounds2_lower.nunique()

**<ins><font size = 4>Solution</font></ins>**: 
---

Data for **66,370** distinct companies is present in rounds2

<font size = 2>Based on the attributes of rounds2, unique vaues in company_permalink (unique id of company) column gives us the unique companies in rounds 2.But as can be observed, initially nunique()'s output (90247) didn't match with lower case unique values of comapny_permalink. This was due to the presence of same values in different cases (refer to table 1.1). This maybe due to compilation of data from various sources with different format.</font>

---
**<font size = 4>Question 2. How many unique companies are present in companies?</font>**
--

**<ins>Approach</ins>**: Using nunique() function of Pandas DataFrame to get the unique values in each column of the dataframe. Also considering data analysis intricacies, unifying the case of the values of the column to lower case and then cross checking for unique values

In [ ]:
companies.nunique()

In [ ]:
companies_lower = companies.copy()
companies_lower['permalink'] =companies_lower['permalink'].str.lower()
companies_lower['name'] =companies_lower['name'].str.lower()
companies[companies.duplicated(['name'],keep=False)].head(7)

**<sup>Table 1.2 Records with same company names but different permalink in companies</sup>**

In [ ]:
companies_lower.nunique()

**<ins><font size = 4>Solution</font></ins>**: 
---

Data for **66,367** distinct companies is present in companies based on the permalink(unique id)

<font size = 2>Based on the attributes of rounds2, unique vaues in permalink (unique id of company) column gives us the unique companies in companies. Another interesting observation is the difference between the distinct permalink(66368) and distinct name of company (66037). Observing in detail we can see  records with same company names but different permalink in companies (refer Table 1.2). This can be due to the update in any of the company attribute which leads to creation of new permalink.For instance:
1. Change in *status* from closed to operating( eg. company name = 3DIVAZ)
2. Change in *homepage_url, category_list, state_code, region, city, founded_at* (eg. Agora)</font>

---
**<font size = 4>Question 3. In the companies data frame, which column can be used as the unique key for each company? Write the name of the column.</font>**
--

**<ins>Approach</ins>**: Checking for the properties of primary key: 

                Check 1. Not Null

                Check 2. Unique for all records


Based on this, column/group of column which uniquesly identifies all rows and is present for all rows should be chosen as a primary key(unique key for each company) 

In [ ]:
# Check 1: 
# Count of non-empty values present in each column
companies_lower.info()

**<ins>Note</ins>**: Only permalink and status column have values for all 66368 rows

In [ ]:
# Check 2:
# Count of non-empty values present in each column
companies_lower.nunique()

**<ins>Note</ins>**: Only permalink column have 6638 uniquevalues for all 66368 rows

**<ins><font size = 4>Solution</font></ins>**:
---
The column named **permalink** passes both the checks and can be used as a primary key to uniquely identify each company.

(Assumption: each record in companies is representing a company)

---
**<font size = 4>Question 4. Are there any companies in the rounds2 file which are not present in companies? Answer yes or no: Y/N</font>**
---

**<ins><font size = 4>Solution</font></ins>**:
---
Y

Yes, as can be seen from the answers of question 1 and 2, data for  66370 unique companies is present in rounds 2 but in companies data for only 66368 unique companies is present. Hence there must be <ins>atleast 2</ins> companies which are present in rounds2 but not in companies. On further analyis we can see there are exactly 7 company permalink whose data is present in rounds2 but not in companies

In [ ]:
rounds2[~rounds2_lower['company_permalink'].isin(companies_lower ['permalink'])]

**<sup>Table 1.3 Showing the permalinks present in rounds but not in companies</sup>**

In [ ]:
#But we can see companies with similar prefix (from table 1.3) in companies
companies[companies.permalink.str.upper().str.contains("/ORGANIZATION/MAGNET-TECH-")]

---
<font size = 4>Question 5. Merge the two data frames so that all variables (columns) in the companies frame are added to the rounds2 data frame. Name the merged frame master_frame. How many observations are present in master_frame?</font>
---

**<ins>Approach</ins>**: For each dataframe, use the dataframe with lower case values of primary key column (company_permalink for rounds2 and permalink for companies) and join them to master_frame. As investment data irrespective of the company is important but companies with no investment are not crucial for this analysis, using logic rounds2 <ins>left join</ins> companies

In [ ]:
master_frame = rounds2_lower.merge(companies_lower,how='left',left_on=['company_permalink'],right_on=['permalink'] )
master_frame.head()

In [ ]:
master_frame.info()

**<font size = 4><ins>Solution</ins></font>**:
---
There are **1,14,949** observations present in the master_frame

**<font size = 4>Checkpoint 1: Expected Results</font>**

|S.No|Questions| Answer     |
|:---:|:------------|:-------------:| 
|1.|How many unique companies are present in rounds2?       | **66,370** |
|2.|How many unique companies are<br/> present in the companies file?  | **66,367**|
|3.|In the companies data frame, which column can be used <br/>as the unique key for each company? <br/>Write the name of the column.|**permalink** |
|4.|Are there any companies in the rounds2 file<br/> which are not  present in companies ? |**Yes**|
|5.|Merge the two data frames so that all  variables (columns)<br/> in the companies frame are added to the rounds2 data frame<br/>Name the merged frame master_frame. <br/> How many observations are present in master_frame ?|**1,14,949**|

___
#  Checkpoint 2: Funding Type Analysis
---

In [ ]:
print(master_frame['funding_round_type'].unique())

*Reference*:The funding types such as seed, venture, angel, etc. depend on the type of the company (startup, corporate, etc.), its stage (early stage startup, funded startup, etc.), the amount of funding *(a few million USD to a billion USD)*, and so on. For example, seed, angel and venture are three common stages of startup funding.

*Seed/angel* funding refer to *early stage* startups whereas *venture* funding occurs after seed or angel stage/s and involves a *relatively higher amount of investment*.
*Private equity type investments* are associated with much larger companies and involve *much higher investments than venture type*. Startups which have grown in scale may also receive private equity funding. This means that if a company has reached the venture stage, it would have already passed through the angel or seed stage/s.

**<ins>Note</ins>**: funding hierarchy is as follow - seed | angel < venture < private equity

___
### <font siz=4>2.1 Calculate the most representative value of the investment amount for each of the four funding types (venture, angel, seed, and private equity) and report the answers 

In [ ]:
# subsetting and checking the distribution and nulls for the four funding types
selected_funding_types = ['venture','angel','seed','private_equity']
selected_funds_master_frame = master_frame[(master_frame['funding_round_type']==selected_funding_types[0])|(master_frame['funding_round_type']==selected_funding_types[1])|(master_frame['funding_round_type']==selected_funding_types[2])|(master_frame['funding_round_type']==selected_funding_types[3])]

In [ ]:
selected_funds_master_frame.info()

In [ ]:
#as we can see some nulls in the above results, calculating the number of nulls for each funding type
print("count of non-empty values of investments by funding type:")
print(selected_funds_master_frame.groupby(by='funding_round_type').agg({'funding_round_type':['count']}))
print("count of nulls values of investments by funding type:")
selected_funds_master_frame[selected_funds_master_frame['raised_amount_usd'].isnull()].groupby(by='funding_round_type').agg({'funding_round_type':['count']})

In [ ]:
#As we know that the amount of funding for each of these types varies significantly in general, without imputing the nulls we will miss
# a significant amount of that funding type which is important for analysis
#To understand the distribution better, plotting bar graph
sns.set(style="whitegrid")
boxplot_1 = sns.boxplot(x=selected_funds_master_frame.funding_round_type, y=selected_funds_master_frame.raised_amount_usd)
boxplot_1

In [ ]:
#checking distribution of each funding type of importance
fig, axes = plt.subplots(ncols=2, nrows=2,figsize=(7, 7))
warnings.filterwarnings("ignore")
for i, ax in zip(range(0,len(selected_funding_types)), axes.flat):
    #print(selected_funding_types[i])
    funding = selected_funds_master_frame[selected_funds_master_frame['funding_round_type']==selected_funding_types[i]]
    funding = funding[~funding['raised_amount_usd'].isnull()]
    x = funding['raised_amount_usd'].astype(np.int16)
    sns.distplot(x,bins=7,ax=ax)
    ax.title.set_text(selected_funding_types[i])
    
plt.tight_layout()
plt.show()


*Plot: ditribution of investment by funding type*

In [ ]:
# Taking into account the outliers present in data, median is more suitable value to fill the nulls
# Hence using median of the funding type by country to impute the values, it is the most representative value of distribution
median_lookup = selected_funds_master_frame
median_lookup= median_lookup.groupby(['funding_round_type', 'country_code']).median()
median_lookup.reset_index(inplace=True)
null_investment_records_df = selected_funds_master_frame[selected_funds_master_frame.raised_amount_usd.isnull()]
#without null in investment
investment_records_df = selected_funds_master_frame[selected_funds_master_frame.raised_amount_usd.notnull()]
# Now we will try to impute the values in master_amt_null
imputed_null_df = null_investment_records_df.merge(median_lookup, on= ['funding_round_type','country_code'] ,how = 'left')
#Normalizing and concatenating the imputed nulls and not null records together
imputed_null_df = imputed_null_df.drop(columns= ['raised_amount_usd_x'])
imputed_null_df = imputed_null_df.rename(columns= {"raised_amount_usd_y" : "raised_amount_usd"} )
imputed_master_df = pd.concat([investment_records_df,imputed_null_df])
imputed_master_df.head()

In [ ]:
imputed_master_df.info()

In [ ]:
#Verification: As can be most of the nulls in the investment amount are filled (exceptions are countries with no relevant data for the particular funding type)
# Main check: after imputing with the nulls, the distribution of data has not changed
df =imputed_master_df
fig, axes = plt.subplots(ncols=2, nrows=2,figsize=(7, 7))
warnings.filterwarnings("ignore")
for i, ax in zip(range(0,len(selected_funding_types)), axes.flat):
    #print(selected_funding_types[i])
    funding = df[df['funding_round_type']==selected_funding_types[i]]
    funding = funding[~funding['raised_amount_usd'].isnull()]
    x = funding['raised_amount_usd'].astype(np.int16)
    sns.distplot(x,bins=7,ax=ax)
    ax.title.set_text(selected_funding_types[i])
    
plt.tight_layout()
plt.show()
imputed_selected_funds_master_frame =df

*Plot: ditribution of investment by funding type*

In [ ]:
investment_by_funding_type_grouped = imputed_selected_funds_master_frame.groupby(['funding_round_type','country_code'])
investment_by_funding_type = investment_by_funding_type_grouped.agg({'raised_amount_usd': [np.mean,np.median,np.max,np.min]})
investment_by_funding_type.sort_values(by=[('raised_amount_usd','mean')],ascending = False)
investment_by_funding_type.head()

In [ ]:
investment_by_funding_type_grouped =imputed_selected_funds_master_frame.groupby(['funding_round_type'])
investment_by_funding_type = investment_by_funding_type_grouped.agg({'raised_amount_usd': ['mean','median','max','min','count']})
investment_by_funding_type.sort_values(by=[('raised_amount_usd','mean')],ascending = False)
investment_by_funding_type

In [ ]:
investment_by_funding_type.loc['venture',:]

In [ ]:
# checking the distribution of investments per round having values between 5 to 15 million
investment_limit_lower = 5000000
investment_limit_upper = 15000000
funds_master_frame=imputed_selected_funds_master_frame[(imputed_selected_funds_master_frame['raised_amount_usd']>=investment_limit_lower)&(imputed_selected_funds_master_frame['raised_amount_usd']<=investment_limit_upper)]
sns.set(style="whitegrid")
boxplot_2 = sns.boxplot(x=funds_master_frame.funding_round_type, y=funds_master_frame.raised_amount_usd)
boxplot_2

___
### 2.2 Based on the most representative investment amount calculated above, which investment type do you think is the most suitable for Spark Funds?

Considering that Spark Funds wants to invest between 5 to 15 million USD per investment round, which investment type is the most suitable for it? Identify the investment type and, for further analysis, filter the data so it only contains the chosen investment type.

**<font size = 4><ins>Solution</ins></font>**:
---
Going by the mean values of investment in USD, only the overall mean (11,843,872.72 USD) of **venture** funding type lies within 5 to 15 million USD per investment round and it has dense distribution in the range of 5-15 million USD

In [ ]:
for fund_type in selected_funding_types:
    print(fund_type,end=': ')
    print( investment_by_funding_type.loc[fund_type,('raised_amount_usd','mean')])

**<font size = 4>Checkpoint 2: Expected Results</font>**

|S.no.|Questions | Initial values (in USD)|After imputing nulls (in USD)
|:---:|:--------:|:------:|:------:|
|1.|Average funding amount of venture type|**11,748,949.13**|**11,227,199.21**|
|2.|Average funding amount of angel type  |**958,694.47**|**868,654.10**|
|3.|Average funding amount of seed type   |**719,818.00**|**655,354.88**|
|4.|Average funding amount of private equity type|**73,308,593.03**|**67,502,962.62**|
|5.|Considering that Spark Funds wants to invest<br/>between 5 to 15 million USD per  investment<br/>round, which investment type is the most <br/>suitable for them?|**venture**|**venture**|


___
#  Checkpoint 3: Country Analysis
---

Spark Funds wants to invest in countries with the highest amount of funding for the chosen investment type. This is a part of its broader strategy to invest where most investments are occurring.

---
### <font size=4>3.1 Spark Funds wants to see the top nine countries which have received the highest total funding (across ALL sectors for the chosen investment type)


In [ ]:
chosen_investment_type = 'venture'
# taking only the venture investment from master frame
selected_master_frame = imputed_selected_funds_master_frame[imputed_selected_funds_master_frame['funding_round_type']==chosen_investment_type]
print(selected_master_frame['funding_round_type'].unique())
print(selected_master_frame['country_code'].unique())

In [ ]:
#
investment_by_country = selected_master_frame.groupby(['country_code'])
investment_by_country = investment_by_country.agg({'raised_amount_usd': ['sum','max','min','count']})
investment_by_country = investment_by_country.sort_values(by=[('raised_amount_usd','sum')],ascending = False)
investment_by_country.head()

---
### <font size =4>3.2 For the chosen investment type, make a data frame named top9 with the top nine countries (based on the total investment amount each country has received)

In [ ]:
# choosing top 9 by total sum of investment amount
top9= investment_by_country.nlargest(9,[('raised_amount_usd','sum')])

In [ ]:
top9

___
### <font size =4> 3.3 Identify the top three English-speaking countries in the data frame top9. 

Iterating over the top 9 df in decreasing order of investment and checking if english spoken there:<br/>
reference : https://en.wikipedia.org/wiki/List_of_territorial_entities_where_English_is_an_official_language
    
    USA : English is a de facto official language
    
    CHN : No Data (Hence non-english speaking country)
    
    GBR : English is a de facto official language
    
    IND : English is a de jure and de facto official language

**<font size = 4>Checkpoint 3: Expected Results</font>**

|S.no.|Questions | Answers|
|:---:|:--------:|:------:|
 |1. |Top English-speaking country	|**USA**|              
 |2. |Second English-speaking country|**GBR**|	 
 |3. |Third English-speaking country |**IND**| 

___
#  Checkpoint 4: Sector Analysis 1
---

When we say sector analysis, we refer to one of the eight main sectors (named main_sector) listed in the mapping file (note that ‘Other’ is one of the eight main sectors). This is to simplify the analysis by grouping the numerous category lists (named ‘category_list’) in the mapping file. For example, in the mapping file, category_lists such as ‘3D’, ‘3D Printing’, ‘3D Technology’, etc. are mapped to the main sector ‘Manufacturing’.

Also, for some companies, the category list is a list of multiple sub-sectors separated by a pipe (vertical bar |). For example, one of the companies’ category_list is Application Platforms|Real Time|Social Network Media.
 

You discuss with the CEO and come up with the business rule that the first string before the vertical bar will be considered the primary sector. In the example above, ‘Application Platforms’ will be considered the primary sector.

In [ ]:
sector_mapping_df = pd.read_csv(mapping_path, encoding="ISO-8859-1")
# checked No category belong to more than 1 main category
print(sector_mapping_df[sector_mapping_df.sum(axis=1)>1])
# main category wise the count of sub categories present
print(sector_mapping_df.sum(axis=0))
sector_mapping_df.head()

---
### <font size=4>4.1 Extract the primary sector of each category list from the category_list column

In [ ]:
selected_master_frame.dtypes

In [ ]:
# correcting the dtypes
selected_master_frame.astype({
    'company_permalink': 'str',
    'funding_round_permalink': 'str',
    'funding_round_type': 'str',
    'funding_round_code': 'str',
   'funded_at':'datetime64[ns]',
    'category_list':'str',
    'status':'str',
    'country_code':'str',
'state_code':'str',
'region':'str',
'city':'str',
'founded_at':'str'   
}).dtypes

In [ ]:
selected_master_frame.head()

In [ ]:
primary_category_fn = lambda category_list: str(category_list).split('|')[0]
selected_master_frame['primary_sector'] = selected_master_frame['category_list'].apply(primary_category_fn)
selected_master_frame[['category_list','primary_sector']].head()

In [ ]:
# reforming mapping df
sector_mapping_df_melted = sector_mapping_df.melt(id_vars="category_list",var_name="main_sector")
sector_mapping_df_melted = sector_mapping_df_melted[sector_mapping_df_melted["value"]>0]
sector_mapping_df_melted=sector_mapping_df_melted.drop(columns="value")
sector_mapping_df_melted.category_list.unique()

In [ ]:
#As observed in the category list in place of string 'na' in the words, '0' can be seen 
# correction: As values like Enterprise 2.0 also contains 0, using regular expression
correction = lambda x: str(x).replace("0","na") if (re.search("[^0-9\.]0",str(x)) != None)  or (re.search("\d\.0",str(x)) == None) else str(x)
sector_mapping_df_melted['category_list'] = sector_mapping_df_melted['category_list'].apply(correction).str.lower()
sector_mapping_df_melted['category_list'].unique()

In [ ]:
selected_master_frame['primary_sector']= selected_master_frame['primary_sector'].str.lower()
master_frame_with_main_sector = selected_master_frame.merge(sector_mapping_df_melted,how='left',left_on='primary_sector',right_on='category_list',suffixes=('_frm_master','_frm_mapping'))

In [ ]:
master_frame_with_main_sector.head()

In [ ]:
master_frame_with_main_sector[(master_frame_with_main_sector['main_sector'].isnull())]['primary_sector'].unique()

**<font size = 4>Checkpoint 4: Expected Results</font>**

In [ ]:
#Code for a merged data frame with each primary sector mapped to its main sector (the primary sector should be present in a separate column).
master_frame_with_main_sector
master_frame_with_main_sector.info()

___
#  Checkpoint 5: Sector Analysis 2
---

Now you have a data frame with each company’s main sector (main_sector) mapped to it. When we say sector analysis, we refer to one of the *<ins>eight main sectors</ins>*.

Also, you know the top three English speaking countries and the most suitable funding type for Spark Funds. Let’s call the three countries 'Country 1', 'Country 2' and 'Country 3' and the funding type 'FT'.

 

Also, the range of funding preferred by Spark Funds is 5 to 15 million USD.

 

Now, the aim is to **find out the most heavily invested main sectors in each of the three countries (for funding type FT and investments range of 5-15 M USD).**

In [ ]:
# Initializing variables based on previous answers
Country_1 = 'USA'
Country_2 = 'GBR'
Country_3 = 'IND'
selected_countries = [Country_1,Country_2,Country_3]
FT = 'venture'
Range_lower = 5000000
Range_upper = 15000000

In [ ]:
#master_frame.info()

In [ ]:
c5_master_frame = master_frame_with_main_sector[(master_frame_with_main_sector['country_code'] == Country_1) | (master_frame_with_main_sector['country_code'] == Country_2) | (master_frame_with_main_sector['country_code'] == Country_3)]
c5_master_frame = c5_master_frame[c5_master_frame['funding_round_type']== FT]
c5_master_frame = c5_master_frame[(c5_master_frame['raised_amount_usd']>=Range_lower) & (c5_master_frame['raised_amount_usd']<=Range_upper)]

In [ ]:
# verifying
print(c5_master_frame['country_code'].unique())
print(c5_master_frame['funding_round_type'].unique())
c5_master_frame['raised_amount_usd'].describe()

In [ ]:
investment_by_sectors = c5_master_frame.groupby(by=['country_code','main_sector'])
investment_by_sectors =investment_by_sectors.agg({'raised_amount_usd': ['sum','count','mean','max','min']})
investment_by_sectors.sort_values(by=[('raised_amount_usd','sum')],ascending=False)

___
### 5.1 Create three separate data frames D1, D2 and D3 for each of the three countries containing the observations of funding type FT falling within the 5-15 million USD range. 
The three data frames should contain:

1. All the columns of the master_frame along with the primary sector and the main sector

2. The total number (or count) of investments for each main sector in a separate column

3. The total amount invested in each main sector in a separate column

In [ ]:
# D1
# country_1 = 'USA'
usa_df = c5_master_frame[c5_master_frame['country_code']=='USA']
#count and sum by main sector
usa_agg = usa_df.groupby('main_sector').agg({"raised_amount_usd": ["count","sum"]}).sort_values(("raised_amount_usd","sum"), ascending = False)
usa_agg

In [ ]:
D1 = usa_df.merge(usa_agg,how='left', on='main_sector')
D1.rename(columns = {('raised_amount_usd', 'count'):'total_no_of_investment_by_sector',
                    ('raised_amount_usd', 'sum'): 'total_amount_invested_by_sector'}, inplace = True)
D1

In [ ]:
# D2
# country_2 = 'GBR'
gbr_df = c5_master_frame[c5_master_frame['country_code']=='GBR']
#count and sum by main sector
gbr_agg = gbr_df.groupby('main_sector').agg({"raised_amount_usd": ["count","sum"]}).sort_values(("raised_amount_usd","sum"), ascending = False)
gbr_agg

In [ ]:
D2 = gbr_df.merge(gbr_agg,how='left', on='main_sector')
D2.rename(columns = {('raised_amount_usd', 'count'):'total_no_of_investment_by_sector',
                    ('raised_amount_usd', 'sum'): 'total_amount_invested_by_sector'}, inplace = True)
D2

In [ ]:
# D3
# country_2 = 'IND'
ind_df = c5_master_frame[c5_master_frame['country_code']=='IND']
#count and sum by main sector
ind_agg = ind_df.groupby('main_sector').agg({"raised_amount_usd": ["count","sum"]}).sort_values(("raised_amount_usd","sum"), ascending = False)
ind_agg

In [ ]:
D3 = ind_df.merge(ind_agg,how='left', on='main_sector')
D3.rename(columns = {('raised_amount_usd', 'count'):'total_no_of_investment_by_sector',
                    ('raised_amount_usd', 'sum'): 'total_amount_invested_by_sector'}, inplace = True)
D3

___
### 5.2 Using the three data frames, you can calculate the total number and amount of investments in each main sector.

In [ ]:
top_countries = ['USA','GBR','IND']
D = [D1,D2,D3]

---
## Q1. Total number of investments (count)

In [ ]:
for i in range(0,3):
    print('\n'+top_countries[i])
    print(D[i].nunique().funding_round_permalink)

---
## Q2. Total amount of investments 

In [ ]:
for i in range(0,3):
    print('\n'+top_countries[i])
    print(D[i]['raised_amount_usd'].sum())

---
## Q3. Top sector (based on count of investments)

In [ ]:
for i in range(0,3):
    print('\n'+top_countries[i])
    print(D[i][D[i].total_no_of_investment_by_sector == D[i].total_no_of_investment_by_sector.max()]['main_sector'].unique())

---
### Q4-8

In [ ]:
top_3_sector_by_company = {}
for i in range(0,3):
    print('\n\n'+top_countries[i])
    top_3_sector_by_company[top_countries[i]] = D[i][['main_sector','total_no_of_investment_by_sector','total_amount_invested_by_sector']].drop_duplicates().sort_values(by='total_no_of_investment_by_sector',ascending=False).head(3)
    print(top_3_sector_by_company[top_countries[i]])
    

---
### 9. For the top sector count-wise (point 3), which company received the highest investment?

In [ ]:
top_sector_by_count = 'Others'

In [ ]:
print("Top company for the top sector (by count): "+top_sector_by_count)
for i in range(0,3):
    print('\n\n'+top_countries[i])
    top_sector_subset = D[i][D[i]['main_sector']==top_sector_by_count]
    top_sector_subset = top_sector_subset[['company_permalink','raised_amount_usd']]
    top_sector_subset_grouped = top_sector_subset.groupby('company_permalink').sum()
    top_company = top_sector_subset_grouped[top_sector_subset_grouped.raised_amount_usd==top_sector_subset_grouped.raised_amount_usd.max()]
    print(top_company)

---
### 10. For the second-best sector count-wise (point 4), which company received the highest investment?

In [ ]:
second_best_sector_by_count ='Social, Finance, Analytics, Advertising'
print("Top company for the second best sector (by count): "+second_best_sector_by_count)
for i in range(0,3):
    print('\n\n'+top_countries[i])
    sector_subset = D[i][D[i]['main_sector']==second_best_sector_by_count]
    sector_subset = sector_subset[['company_permalink','raised_amount_usd']]
    sector_subset_grouped = sector_subset.groupby('company_permalink').sum()
    top_company = sector_subset_grouped[sector_subset_grouped.raised_amount_usd==sector_subset_grouped.raised_amount_usd.max()]
    print(top_company)

___
#  Checkpoint 6: Plots
---

1. A plot showing the fraction of total investments (globally) in angel, venture, seed, and private equity, and the average amount of investment in each funding type. This chart should make it clear that a certain funding type (FT) is best suited for Spark Funds.

2. A plot showing the top 9 countries against the total amount of investments of funding type FT. This should make the top 3 countries (Country 1, Country 2, and Country 3) very clear.

3. A plot showing the number of investments in the top 3 sectors of the top 3 countries on one chart (for the chosen investment type FT). 

---
## Q1. A plot showing the fraction of total investments (globally) in angel, venture, seed, and private equity, and the average amount of investment in each funding type. This chart should make it clear that a certain funding type (FT) is best suited for Spark Funds.

In [ ]:
#taking global(all countries all sectors) maaster frame with selected funding types
fund_type_df =selected_funds_master_frame.groupby('funding_round_type').agg({'raised_amount_usd':['sum','mean']})
# fund_type_df['overall_total'] = fund_type_df[('raised_amount_usd','sum')].sum()
# fund_type_df['perc_share'] = (fund_type_df[('raised_amount_usd','sum')]/fund_type_df['overall_total'])*100
#fund_type_df.reset_index()
fund_type_df

In [ ]:
share_plot = fund_type_df.plot.pie(y=('raised_amount_usd','sum'),autopct='%.1f%%',legend=True,use_index=True,title='Funding Type by Percentage Share Global', textprops={'fontsize': 14}, figsize=(8.5,8.5))
share_plot.set_xlabel("")
share_plot.set_ylabel("")
share_plot.legend(title="Funding Type")


In [ ]:
avg_plot = fund_type_df[[('raised_amount_usd','mean')]].plot.bar(legend=False,logy=True,title="Average Investment by Fund Type",figsize=(10,7))
avg_plot.set_xlabel("Funding Round Type")
avg_plot.set_ylabel("Average Investment in USD (At log scale)")
avg_plot.axhline( y=5000000, xmin=0, xmax=1,color='k', linestyle='--')
avg_plot.text(-0.20,5000000, "Lower Limit (5 million USD)")
avg_plot.axhline( y=15000000, xmin=0, xmax=1,color='k', linestyle='--')
avg_plot.text(-0.20,15000000, "Upper Limit (15 million USD)")


Assumption: Mean is the  most representative of the data distribution

---
## Q2. A plot showing the top 9 countries against the total amount of investments of funding type FT. This should make the top 3 countries (Country 1, Country 2, and Country 3) very clear.

In [ ]:
top9.head()
#df for checkpoint 6 question 2
c6_q2 = pd.DataFrame(index=top9.index, columns=['sum_invested','english_speaking'])
c6_q2['sum_invested'] = top9[('raised_amount_usd','sum')]
c6_q2['english_speaking'] = ['Yes','No','Yes','Yes','Yes','No','Yes','No','No']
c6_q2

In [ ]:
colors = {'Yes': 'b', 'No': 'r'}
top9_plot = c6_q2['sum_invested'].plot(kind='bar',logy=True,title="Investment by Top 9 countries", color=[colors[i] for i in c6_q2['english_speaking']])
top9_plot.set_xlabel("Top 9 Countries")
top9_plot.set_ylabel("Investment (At log scale)")

##### Plot 2. Investment by countries (red color depicts non-english speaking countries)

Assumptions: 
1. Top countires are one with highest sum invested in selected funding type (Venture)
2. Venture is the most suitable funding type considering the budget    

---
## Q3.A plot showing the number of investments in the top 3 sectors of the top 3 countries on one chart (for the chosen investment type FT).
FT= 'venture'

In [ ]:
#adding country to df
for key,item in top_3_sector_by_company.items():
    top_3_sector_by_company[key]['country_code'] = key


In [ ]:
#concatenating all top3 countries df having top 3 sectors
top_3_sector_by_country_all =  pd.concat([top_3_sector_by_company['USA'], top_3_sector_by_company['GBR'],top_3_sector_by_company['IND']],axis=0)
top_3_sector_by_country_all.head()

In [ ]:
#Plot
top_sectors_plot = sns.barplot(x='country_code', y='total_amount_invested_by_sector', hue='main_sector', data=top_3_sector_by_country_all)
plt.ylabel('Investment in USD (log scale)')
plt.xlabel('country code')
plt.title('Top 3 sectors of Top 3 countries')
# Put the legend out of the figure
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
top_sectors_plot.set_yscale("log")

---
                                                         End of Assignment
---